In [1]:
from Base.MachiKoro_new._env import*

import numpy as np
import numba as nb


from scipy.stats.mstats import gmean, hmean
from numba import njit

In [2]:
# @njit()
def bot_monte(state, perData):
    validActions = getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    action_return = -1
    count_min = 10000
    
    # print('RUN lượt 1')
    for action in arr_action[:1]:
        state_test = state.copy()
        action_test = action.copy()
        # print('CHECK', arr_action, getReward(state_test))
        id_action = 0
        count_change = 0
        count = 0
        while getReward(state_test) == -1:
            count += 1
            
            # else:
                # print(count, ' doing', action_test)
            # print('loop while')
            try:
                arr_next_state, arr_prob_next = getPredictState(state_test, action_test)
            except:
                print(list(state_test))
                raise Exception('toang predict')
            # state_test = arr_next_state[np.random.randint(0, len(arr_prob_next))]       #chọn ngẫu nhiên 1 state
            state_test = arr_next_state[0]
            id_real_action = np.where(state_test[P_ID_ACTION : P_ID_ACTION + NUMBER_PLAYER])[0][0]
            
            if id_real_action != id_action:
                count_change += 1
                id_action = id_real_action

            arr_action_test = np.where(getValidActions(state_test))[0]                  #các action khả dĩ ở state được chọn
            action_test = np.random.choice(arr_action_test)                             #chọn ngẫu nhiên 1 action
            # if state_test[P_CHECK_END] == 1:
            #     print('end game')

        # print('DONE loop')
        count_turn = count_change // 4

        if count_turn < count_min:
            count_min = count_turn
            action_return = action
    # print('check', action_return, count_min)

    return action_return, perData




In [3]:
per = np.zeros(10)

win, per = numba_main_2(bot_monte, 100, per, 0)

print("DONE", win)

In [4]:
state = np.array([26.0, 5.0, 4.0, 3.0, 5.0, 3.0, 3.0, 6.0, 4.0, 6.0, 2.0, 3.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 1.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 2.0, 1.0, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]


)



state[P_CARD_SELL : P_CARD_SELL + NUMBER_TYPE_NORMAL_CARD]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [5]:
state[P_PHASE : P_PHASE + NUMBER_PHASE]

array([0., 0., 0., 0., 0., 1., 0.])

In [7]:
np.where(state[P_CARD_SELL : P_CARD_SELL + NUMBER_TYPE_NORMAL_CARD])[0][0] + 1

8

In [34]:
@njit()
def getPseudoAction(state, per_file):
    real_action = getValidActions(state)

    list_action_return = np.zeros(len(per_file[0]))
    list_action_return[:getActionSize()] = real_action

    for id in range(len(per_file[1])):
        action_real = np.where(per_file[1][id])[0][0]
        check = True
        for id_c in range(len(per_file[1][id])):
            if np.sum(state*per_file[2][id][id_c]) <= per_file[3][id][id_c]:
                check = False
                break
        if check and real_action[action_real] == 1:
            list_action_return[getActionSize() + id] = 1
        
    
    return list_action_return


@njit()
def agent_Hieu(state, per):
    bias = per[0][0][0]
    arr_actions = getPseudoAction(state, per)
    action = np.argmax(bias*arr_actions)
    if action == len(bias) - 2:
        action = 1
    elif action == len(bias) - 1:
        action = 4

    return action, per

from numba.typed import List, Dict

@njit()
def create_perx():
    perx = Dict()

    temp = List([np.zeros((1,500))])
    temp.pop(0)
    perx[0] = temp.copy()       #bias
    perx[1] = temp.copy()       #action real
    perx[2] = temp.copy()       #arr operator (+,-, constant)
    perx[3] = temp.copy()       #value compare
    perx[4] = temp.copy()


    return perx

per = create_perx()
per[0].append(np.random.rand(1,500))

arr_action_pseudo = np.zeros((1,500))
state_condition_operator = np.zeros((2,500))
value_compare = np.zeros((2,1))


arr_action_pseudo[0][1] = 1
state_condition_operator[0][17] = 1
state_condition_operator[1][6] = 1
value_compare[0] = 5
value_compare[1] = 3

per[1].append(arr_action_pseudo)
per[2].append(state_condition_operator)
per[3].append(value_compare)

arr_action_pseudo = np.zeros((1,500))
state_condition_operator = np.zeros((2,500))
value_compare = np.zeros((2,1))

arr_action_pseudo[0][4] = 1
state_condition_operator[0][3] = 1
state_condition_operator[0][5] = -1

state_condition_operator[1][4] = 1
state_condition_operator[1][2] = 1

value_compare[0] = 0
value_compare[1] = 7

per[1].append(arr_action_pseudo)
per[2].append(state_condition_operator)
per[3].append(value_compare)




In [4]:
import numpy as np
from numba import njit

@njit()
def test():
    a = np.zeros((2,15))
    a
    b = np.zeros(15)
    print(b)
    c = b.reshape(1,15)
    a = np.append(a,c, axis=0)
    return a
test()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [36]:
win, per = numba_main_2(agent_Hieu, 1000, per, 0)



ValueError: cannot assign slice from input of different size

In [6]:
b = np.random.randint(1,5,(2,3))
b

array([[1, 2, 3],
       [2, 1, 3]])

In [7]:
a*b

array([[4, 6, 3],
       [8, 2, 9]])

In [9]:
c = np.random.randint(1,5,(1,3))
c

array([[1, 1, 2]])

In [10]:
b*c

array([[1, 2, 6],
       [2, 1, 6]])

In [1]:
-3%4

1